In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.5.0 (SDL 2.28.0, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'C:\Users\shaik\OneDrive\Desktop\Drowsiness-Detection-System\models\model.h5')

In [3]:
mixer.init()
sound = mixer.Sound(r'C:\Users\shaik\OneDrive\Desktop\Drowsiness-Detection-System\alarm.wav')

# Initialize the video capture
cap = cv2.VideoCapture(0)

Score = 0
closed_eyes_count = 0
closed_eyes_threshold = 5
is_alarm_playing = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Reset the Score for each frame
    Score = 0

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    for (x, y, w, h) in faces:
        # Comment out or remove the code below to remove the blue frame around the detected face
        #cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=3)

        # Perform eye detection within the face region
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=1)

        for (ex, ey, ew, eh) in eyes:
            # Preprocess the eye region
            eye = roi_color[ey:ey+eh, ex:ex+ew]
            eye = cv2.resize(eye, (80, 80))
            eye = eye / 255.0
            eye = np.expand_dims(eye, axis=0)

            # Perform prediction using the loaded model
            prediction = model.predict(eye)

            # If eyes are closed
            if prediction[0][0] > 0.30:
                cv2.putText(frame, 'closed', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1,
                            color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                closed_eyes_count += 1
                Score += 1

                if closed_eyes_count > closed_eyes_threshold and not is_alarm_playing:
                    try:
                        sound.play()
                        is_alarm_playing = True
                    except:
                        pass

            # If eyes are open
            elif prediction[0][1] > 0.90:
                cv2.putText(frame, 'open', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1,
                            color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                closed_eyes_count = 0

    # Update the alarm state outside the loop
    if Score <= closed_eyes_threshold and is_alarm_playing:
        try:
            sound.stop()
            is_alarm_playing = False
        except:
            pass

    cv2.putText(frame, 'Score ' + str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 28ms/step
